# MAL Analysis

## Cleaning

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('mal_top_999.csv').drop(columns='Unnamed: 0')
df.head(2)

,Title,Rank,Popularity,Score,Score_Count,Members,Favorites,Type,Episodes,Status,Season,Year,Producers,Licensors,Studios,Genres,Themes,Demographic
0,Fullmetal Alchemist: Brotherhood,1,3,9.11,1968498,3096928,213711,TV,64,Finished Airing,Spring,2009,"['Aniplex', 'Square Enix', 'Mainichi Broadcast...","['Funimation', 'Aniplex of America']",['Bones'],"['Action', 'Adventure', 'Drama', 'Fantasy']",Military,Shounen
1,Bleach: Sennen Kessen-hen,2,518,9.10,176150,395099,15982,TV,13,Finished Airing,Fall,2022,"['TV Tokyo', 'Aniplex', 'Dentsu', 'Shueisha']",['VIZ Media'],['Pierrot'],"['Action', 'Adventure', 'Fantasy']",NaN,Shounen


In [3]:
df_clean = df.copy()
df_clean.loc[df_clean['Episodes'] == 'Unknown', 'Episodes'] = 0
df_clean['Episodes'] = df_clean['Episodes'].astype(int)

df_clean['Producers'] = df_clean['Producers'].apply(lambda x: x[1:-1].replace("'","").replace('add some', '').split(', '))
df_clean['Studios'] = df_clean['Studios'].apply(lambda x: x[1:-1].replace("'","").split(', '))
df_clean['Licensors'] = df_clean[df_clean['Licensors'] != '']['Licensors'].apply(lambda x: x[1:-1].replace("'","").replace('add some', '').split(', '))
df_clean['Themes'] = df_clean[~df_clean['Themes'].isna()]['Themes'].astype(str)
df_clean['Themes'] = df_clean[~df_clean['Themes'].isna()]['Themes'].apply(lambda x: list(map(str, x.replace("'", "").strip('[]').split(', '))) if ',' in x else [x.replace('nan', '')])
df_clean['Genres'] = df_clean[~df_clean['Genres'].isna()]['Genres'].astype(str)
df_clean['Genres'] = df_clean[~df_clean['Genres'].isna()]['Genres'].apply(lambda x: x[1:-1].replace("'","").replace('add some', '').split(', '))

df_clean.loc[df_clean['Licensors'].apply(lambda x: len(x) == 1 and '' in x), 'Licensors'] = df_clean[df_clean['Licensors'].apply(lambda x: len(x) == 1 and '' in x)]['Licensors'].apply(lambda x: np.NaN)
df_clean.loc[df_clean['Producers'].apply(lambda x: len(x) == 1 and '' in x), 'Producers'] = df_clean[df_clean['Producers'].apply(lambda x: len(x) == 1 and '' in x)]['Producers'].apply(lambda x: np.NaN)
# df_clean.replace('', pd.NaT, inplace=True)
df_clean.head(3)


,Title,Rank,Popularity,Score,Score_Count,Members,Favorites,Type,Episodes,Status,Season,Year,Producers,Licensors,Studios,Genres,Themes,Demographic
0,Fullmetal Alchemist: Brotherhood,1,3,9.11,1968498,3096928,213711,TV,64,Finished Airing,Spring,2009,"[Aniplex, Square Enix, Mainichi Broadcasting S...","[Funimation, Aniplex of America]",[Bones],"[Action, Adventure, Drama, Fantasy]",[Military],Shounen
1,Bleach: Sennen Kessen-hen,2,518,9.10,176150,395099,15982,TV,13,Finished Airing,Fall,2022,"[TV Tokyo, Aniplex, Dentsu, Shueisha]",[VIZ Media],[Pierrot],"[Action, Adventure, Fantasy]",NaN,Shounen
2,Steins;Gate,3,13,9.08,1307453,2385176,179860,TV,24,Finished Airing,Spring,2011,"[Frontier Works, Media Factory, Movic, AT-X, K...",[Funimation],[White Fox],"[Drama, Sci-Fi, Suspense]","[Psychological, Time Travel]",NaN


In [4]:
# Exploade to include different values for later studies
df_exploded = ((((df_clean.explode('Producers')).explode('Licensors')).explode('Studios')).explode('Genres')).explode('Themes')



### Studio Specific Information

In [5]:
# Due to abundance of small companies, catgorize these studios as others
df_exploded_studios = df_clean.explode('Studios').reset_index(drop = True)
studio_stats = df_exploded_studios.groupby(['Studios']).size().sort_values(ascending=False).reset_index(drop = False)
small_studio_arrays = studio_stats[studio_stats[0] <= 3]['Studios'].unique()
df_exploded_studios['Studios'] = df_exploded_studios['Studios'].replace(small_studio_arrays, 'Others')
df_exploded_studios['Studios'] = df_exploded_studios['Studios'].replace('add some', 'Others')
# df_exploded_studios.head(3)

df_exploded['Studios'] = df_exploded['Studios'].replace(small_studio_arrays, 'Others')
df_exploded['Studios'] = df_exploded['Studios'].replace('add some', 'Others')
# df_exploded.to_csv('mal-exploded.csv', index=False)


In [6]:
### KEY:
## df_exploded_studs = info of studios
## df_exploded_lic = info of licensors 
## df_exploded_stugen = info of studios + genres
## df_exploded_themes = info of themes
## df_exploded_prod = info of producers
## df_exploded_gens = info of genres
key_cols = np.array(range(12))

df_exploded_studs = df_exploded_studios.copy()
df_exploded_studs = df_exploded_studs.iloc[:,np.append(key_cols, [14,17])]
df_exploded_studs = df_exploded_studs.drop_duplicates()

df_exploded_lic = df_exploded.iloc[:,np.append(key_cols, [13,17])]
df_exploded_lic = df_exploded_lic.drop_duplicates()

df_exploded_stugen = df_exploded.iloc[:,np.append(key_cols, [14,15,17])]
df_exploded_stugen = df_exploded_stugen.drop_duplicates()

df_exploded_themes = df_exploded.iloc[:,np.append(key_cols, [16,17])]
df_exploded_themes = df_exploded_themes.drop_duplicates()

df_exploded_prod = df_exploded.iloc[:,np.append(key_cols, [12,17])]
df_exploded_prod = df_exploded_prod.drop_duplicates()

df_exploded_gens = df_exploded.iloc[:,np.append(key_cols, [15,17])]
df_exploded_gens = df_exploded_gens.drop_duplicates()
# df_exploded_studs.to_csv('df_exploded_studs.csv', index=False)
# df_exploded_lic.to_csv('df_exploded_lic.csv', index=False)
# df_exploded_stugen.to_csv('df_exploded_stugen.csv', index=False)
# df_exploded_themes.to_csv('df_exploded_themes.csv', index=False)
# df_exploded_prod.to_csv('df_exploded_prod.csv', index=False)
# df_exploded_gens.to_csv('df_exploded_gens.csv', index=False)

In [28]:
mal_ranks = df_clean.copy()
mal_ranks.head(2)

,Title,Rank,Popularity,Score,Score_Count,Members,Favorites,Type,Episodes,Status,Season,Year,Producers,Licensors,Studios,Genres,Themes,Demographic
0,Fullmetal Alchemist: Brotherhood,1,3,9.11,1968498,3096928,213711,TV,64,Finished Airing,Spring,2009,"[Aniplex, Square Enix, Mainichi Broadcasting S...","[Funimation, Aniplex of America]",[Bones],"[Action, Adventure, Drama, Fantasy]",[Military],Shounen
1,Bleach: Sennen Kessen-hen,2,518,9.10,176150,395099,15982,TV,13,Finished Airing,Fall,2022,"[TV Tokyo, Aniplex, Dentsu, Shueisha]",[VIZ Media],[Pierrot],"[Action, Adventure, Fantasy]",NaN,Shounen


In [31]:
mal_ranks['Diff'] = np.abs(mal_ranks['Rank'] - mal_ranks['Popularity'])
mal_ranks['Dedication'] = mal_ranks["Score_Count"]/mal_ranks['Members']

In [69]:
mal_ranks = mal_ranks.sort_values('Dedication')
mal_ranks[(mal_ranks['Popularity'] > 2000) & (mal_ranks['Type'] == 'TV') & (mal_ranks['Dedication'] > 0.4) & (mal_ranks['Score'] > 8)].sort_values('Diff', ascending = False).tail(20)
# mal_ranks[(mal_ranks['Type'] == 'TV') & (mal_ranks['Popularity']>500)].tail(20)

,Title,Rank,Popularity,Score,Score_Count,Members,Favorites,Type,Episodes,Status,Season,Year,Producers,Licensors,Studios,Genres,Themes,Demographic,Diff,Dedication
574,Pui Pui Molcar,576,5088,8.01,8687,14514,252,TV,12,Finished Airing,Winter,2021,NaN,[Muse Communication],[Shin-Ei Animation],NaN,"[Anthropomorphic, Racing]",Kids,4512,0.598526
437,Kidou Senshi Gundam: The Origin - Zenya Akai S...,438,4544,8.12,8578,19454,95,TV,13,Finished Airing,Spring,2019,[Sunrise Music],NaN,[Sunrise],"[Action, Sci-Fi]","[Mecha, Military, Space]",Shounen,4106,0.440938
338,Tanoshii Muumin Ikka,339,4085,8.21,11182,24729,637,TV,78,Finished Airing,Spring,1990,"[TV Tokyo, Starchild Records]",NaN,[Telescreen],"[Adventure, Comedy, Fantasy, Slice of Life]",[Iyashikei],Kids,3746,0.452182
504,City Hunter 2,506,4064,8.06,10407,24950,153,TV,63,Finished Airing,Spring,1988,NaN,"[ADV Films, Discotek Media]",[Sunrise],"[Action, Comedy, Mystery]",NaN,Shounen,3558,0.417114
564,Ginga Nagareboshi Gin,566,4008,8.02,11026,25645,805,TV,21,Finished Airing,Spring,1986,[TV Asahi],NaN,[Toei Animation],"[Action, Adventure, Drama]",[Anthropomorphic],Shounen,3442,0.429947
536,BanG Dream! 3rd Season,538,3530,8.03,14652,33014,567,TV,13,Finished Airing,Winter,2020,"[Ultra Super Pictures, Bushiroad, Tokyo MX, Go...",[Sentai Filmworks],[SANZIGEN],NaN,[Music],NaN,2992,0.443812
531,Saiunkoku Monogatari 2nd Season,533,3228,8.03,17441,39511,199,TV,39,Finished Airing,Spring,2007,"[NHK, Sogo Vision]",NaN,[Madhouse],"[Adventure, Fantasy, Romance]","[Historical, Reverse Harem]",Shoujo,2695,0.441421
548,Uma Musume: Pretty Derby Season 2,550,3100,8.03,17479,42497,668,TV,13,Finished Airing,Winter,2021,"[TOHO animation, Cygames]",NaN,[Studio Kai],NaN,[Anthropomorphic],NaN,2550,0.411300
208,Major S6,209,2569,8.33,35305,60164,472,TV,25,Finished Airing,Spring,2010,"[Shogakukan-Shueisha Productions, NHK]",NaN,[SynergySP],NaN,[Team Sports],Shounen,2360,0.586813
167,Major S5,168,2389,8.41,39830,68068,618,TV,25,Finished Airing,Winter,2009,"[Shogakukan-Shueisha Productions, NHK]",NaN,[SynergySP],NaN,[Team Sports],Shounen,2221,0.585150
